In [8]:
import json
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from shapely.geometry import shape
import pandas as pd

In [10]:
precincts=json.load(open('first-phase/New York/New York Police Precincts.geojson','r'))
# precincts=precincts['features']
# print(len(precincts))

polygons=[]
# precincts[0]
cx,cy=0,0
for precinct in precincts['features']:
  precinct['precinct']=precinct['properties']['precinct']
  polygons.append([shape(precinct['geometry']),precinct['properties']['precinct']])

long=0
lat=0
for p in polygons:
  cord=p[0].centroid.coords[0]
  long+=cord[0]
  lat+=cord[1]
long/=len(polygons)
lat/=len(polygons)
with open('./new-york-city.json','w') as file:
  json.dump({
    "latitude": lat, 
    "longitude": long,
  },file)
# precincts['features']
# polygons[0][0].centroid.coords[0]

In [11]:
df=pd.read_csv('./first-phase/Seattle/Seattle Db.csv')
prev_crime_data=pd.read_csv('./seattle-precinct-wise-dataset.csv')
df=df[10001:20000]
list1=[]
for polygon in polygons:
  c=0
  count={'precinct': polygon[1],'total':0}
  # count=0
  for index,row in df.iterrows():
    if polygon[0].contains(Point(row['Longitude'],row['Latitude'])):
      if row['Offence'] not in count:
        count[row['Offence']]=0
      count[row['Offence']]+=1
      count['total']+=1
      break
      # df.drop(index)
  list1.append(count)

In [ ]:
# list1
# crime_data=pd.DataFrame.from_dict(list1)
# print(prev_crime_data.head(1))
# print(crime_data.head(1))
# x=crime_data.head(1).add(prev_crime_data.head(1))
# x
prev_crime_data.to_csv('seattle-precinct-wise-dataset.csv')

In [5]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default='chrome'
crime_data=pd.read_csv('seattle-precinct-wise-dataset.csv')
data=json.load(open('./seattle.json'))
lat=data['latitude']
long=data['longitude']

In [6]:
fig=px.choropleth_mapbox(crime_data,locations='precinct',featureidkey="properties.precinct",geojson=precincts,color='total',color_continuous_scale='ylorrd')
# fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(mapbox_style="carto-positron", 
                  mapbox_zoom=9.5,
                  mapbox_center={"lat": lat, "lon": long},
                  margin={"r":0,"t":0,"l":0,"b":0},
                  uirevision='constant')
fig.write_html('first_figure-3.html')